In [1]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import skimage.io as io
import os, sys
from matplotlib import pyplot as plt

# Use second GPU -- change if you want to use a first one
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

# Add a path to a custom fork of TF-Slim
# Get it from here:
# https://github.com/warmspringwinds/models/tree/fully_conv_vgg
sys.path.append("/home/lennon.lin/Repository/github/tensorflow-Model/slim/")

# Add path to the cloned library
sys.path.append("/home/lennon.lin/Repository/github/tf-image-segmentation/")

checkpoints_dir = '/home/lennon.lin/Checkpoint'
log_folder = '/home/lennon.lin/Repository/github/cityscape-segmentation/log_folder_resnet_101_16s'

number_of_epochs = 20

slim = tf.contrib.slim
resnet_101_v1_checkpoint_path = os.path.join(checkpoints_dir, 'resnet_v1_101.ckpt')

from tf_image_segmentation.utils.tf_records import read_tfrecord_and_decode_into_image_annotation_pair_tensors
from tf_image_segmentation.models.resnet_v1_101_16s import resnet_v1_101_16s, extract_resnet_v1_101_mapping_without_logits

from tf_image_segmentation.utils.pascal_voc import pascal_segmentation_lut
from tf_image_segmentation.utils.cityscape import cityscape_Label

from tf_image_segmentation.utils.training import get_valid_logits_and_labels

from tf_image_segmentation.utils.augmentation import (distort_randomly_image_color,
                                                      flip_randomly_left_right_image_with_annotation,
                                                      scale_randomly_image_with_annotation_with_fixed_size_output)

from tensorflow.python.ops import control_flow_ops

image_train_size = [1024, 1024]
#number_of_classes = 21
tfrecord_filename = '/home/lennon.lin/Repository/github/tf-image-segmentation/tf_image_segmentation/recipes/cityscape/cityscape_trainid_train.tfrecords'
#pascal_voc_lut = pascal_segmentation_lut()
#class_labels = pascal_voc_lut.keys()
#class_labels = map(lambda x: x[2] ,cityscape_Label)
class_labels = [0, 1, -1]
number_of_classes = 2
print(class_labels)

[0, 1, -1]


In [ ]:
filename_queue = tf.train.string_input_producer(
    [tfrecord_filename], num_epochs=number_of_epochs)

image, annotation = read_tfrecord_and_decode_into_image_annotation_pair_tensors(filename_queue)

# Various data augmentation stages
image, annotation = flip_randomly_left_right_image_with_annotation(image, annotation)

# image = distort_randomly_image_color(image)

resized_image, resized_annotation1 = scale_randomly_image_with_annotation_with_fixed_size_output(image, annotation, image_train_size)


resized_annotation = tf.squeeze(resized_annotation1)


image_batch, annotation_batch = tf.train.shuffle_batch( [resized_image, resized_annotation],
                                             batch_size=1,
                                             capacity=3000,
                                             num_threads=2,
                                             min_after_dequeue=1000)

tf.summary.image("training image",image_batch)

# is_training=False here means that we fix the mean and variance
# of original model that was trained on Imagenet.
# https://github.com/tensorflow/tensorflow/issues/1122
upsampled_logits_batch, resnet_v1_101_variables_mapping = resnet_v1_101_16s(image_batch_tensor=image_batch,
                                                           number_of_classes=number_of_classes,
                                                           is_training=True)


valid_labels_batch_tensor, valid_logits_batch_tensor = get_valid_logits_and_labels(annotation_batch_tensor=annotation_batch,
                                                                                     logits_batch_tensor=upsampled_logits_batch,
                                                                                    class_labels=class_labels)



Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
INFO:tensorflow:Summary name training image is illegal; using training_image instead.


In [ ]:
cross_entropies = tf.nn.softmax_cross_entropy_with_logits(logits=valid_logits_batch_tensor,
                                                          labels=valid_labels_batch_tensor)

# Normalize the cross entropy -- the number of elements
# is different during each step due to mask out regions
cross_entropy_sum = tf.reduce_mean(cross_entropies)

pred = tf.argmax(upsampled_logits_batch, dimension=3)

probabilities = tf.nn.softmax(upsampled_logits_batch)


with tf.variable_scope("adam_vars"):
    train_step = tf.train.AdamOptimizer(learning_rate=0.000001).minimize(cross_entropy_sum)


# Variable's initialization functions
resnet_v1_101_without_logits_variables_mapping = extract_resnet_v1_101_mapping_without_logits(resnet_v1_101_variables_mapping)


init_fn = slim.assign_from_checkpoint_fn(model_path=resnet_101_v1_checkpoint_path,
                                         var_list=resnet_v1_101_without_logits_variables_mapping)

global_vars_init_op = tf.global_variables_initializer()

tf.summary.scalar('cross_entropy_loss', cross_entropy_sum)

merged_summary_op = tf.summary.merge_all()

summary_string_writer = tf.summary.FileWriter(log_folder)

# Create the log folder if doesn't exist yet
if not os.path.exists(log_folder):
     os.makedirs(log_folder)
    
#The op for initializing the variables.
local_vars_init_op = tf.local_variables_initializer()

combined_op = tf.group(local_vars_init_op, global_vars_init_op)

# We need this to save only model variables and omit
# optimization-related and other variables.
model_variables = slim.get_model_variables()
saver = tf.train.Saver(model_variables)


with tf.Session()  as sess:
    
    sess.run(combined_op)
    init_fn(sess)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # 10 epochs
    for i in xrange(11127 * number_of_epochs):
    
        cross_entropy, summary_string, _ = sess.run([ cross_entropy_sum,
                                                      merged_summary_op,
                                                      train_step
                                                      ])
        
        print("step" + str(i) + ", Current loss: " + str(cross_entropy))
        
        summary_string_writer.add_summary(summary_string, i)
        
        if i % 11127 == 0:
            save_path = saver.save(sess, "/home/lennon.lin/Repository/github/cityscape-segmentation/model_resnet_101_16s.ckpt")
            print("Model saved in file: %s" % save_path)
            
        
    coord.request_stop()
    coord.join(threads)
    
    save_path = saver.save(sess, "/home/lennon.lin/Repository/github/cityscape-segmentation/model_resnet_101_16s.ckpt")
    print("Model saved in file: %s" % save_path)
    
summary_string_writer.close()

INFO:tensorflow:Restoring parameters from /home/lennon.lin/Checkpoint/resnet_v1_101.ckpt
step0, Current loss: 0.69417
Model saved in file: /home/lennon.lin/Repository/github/cityscape-segmentation/model_resnet_101_16s.ckpt
step1, Current loss: 0.648653
step2, Current loss: 0.653709
step3, Current loss: 0.615717
step4, Current loss: 0.479
step5, Current loss: 0.603548
step6, Current loss: 0.557762
step7, Current loss: 0.544602
step8, Current loss: 0.569553
step9, Current loss: 0.425976
step10, Current loss: 0.530935
step11, Current loss: 0.558481
step12, Current loss: 0.509514
step13, Current loss: 0.484681
step14, Current loss: 0.605607
step15, Current loss: 0.441349
step16, Current loss: 0.529824
step17, Current loss: 0.545628
step18, Current loss: 0.632062
step19, Current loss: 0.536097
step20, Current loss: 0.400706
step21, Current loss: 0.357047
step22, Current loss: 0.420321
step23, Current loss: 0.559606
step24, Current loss: 0.578257
step25, Current loss: 0.558712
step26, Curren

step255, Current loss: 0.268656
step256, Current loss: 0.162685
step257, Current loss: 0.196425
step258, Current loss: 0.181713
step259, Current loss: 0.202227
step260, Current loss: 0.204489
step261, Current loss: 0.184606
step262, Current loss: 0.350371
step263, Current loss: 0.24037
step264, Current loss: 0.238328
step265, Current loss: 0.315064
step266, Current loss: 0.275622
step267, Current loss: 0.246611
step268, Current loss: 0.207688
step269, Current loss: 0.248197
step270, Current loss: 0.364212
step271, Current loss: 0.245641
step272, Current loss: 0.27019
step273, Current loss: 0.224306
step274, Current loss: 0.242361
step275, Current loss: 0.134331
step276, Current loss: 0.172465
step277, Current loss: 0.198428
step278, Current loss: 0.254125
step279, Current loss: 0.254872
step280, Current loss: 0.163414
step281, Current loss: 0.311783
step282, Current loss: 0.244209
step283, Current loss: 0.159554
step284, Current loss: 0.153191
step285, Current loss: 0.241418
step286, C

step512, Current loss: 0.147838
step513, Current loss: 0.144714
step514, Current loss: 0.190048
step515, Current loss: 0.220689
step516, Current loss: 0.220522
step517, Current loss: 0.165592
step518, Current loss: 0.195725
step519, Current loss: 0.131033
step520, Current loss: 0.188689
step521, Current loss: 0.179762
step522, Current loss: 0.139015
step523, Current loss: 0.227434
step524, Current loss: 0.182648
step525, Current loss: 0.112516
step526, Current loss: 0.167481
step527, Current loss: 0.24687
step528, Current loss: 0.167888
step529, Current loss: 0.153066
step530, Current loss: 0.178766
step531, Current loss: 0.145688
step532, Current loss: 0.14447
step533, Current loss: 0.259047
step534, Current loss: 0.130588
step535, Current loss: 0.180167
step536, Current loss: 0.142052
step537, Current loss: 0.178917
step538, Current loss: 0.202898
step539, Current loss: 0.172969
step540, Current loss: 0.137097
step541, Current loss: 0.272411
step542, Current loss: 0.181902
step543, C

step770, Current loss: 0.197059
step771, Current loss: 0.161543
step772, Current loss: 0.189509
step773, Current loss: 0.169313
step774, Current loss: 0.143547
step775, Current loss: 0.191044
step776, Current loss: 0.157508
step777, Current loss: 0.166616
step778, Current loss: 0.16422
step779, Current loss: 0.222875
step780, Current loss: 0.166095
step781, Current loss: 0.205326
step782, Current loss: 0.229298
step783, Current loss: 0.168645
step784, Current loss: 0.24712
step785, Current loss: 0.227832
step786, Current loss: 0.195049
step787, Current loss: 0.140884
step788, Current loss: 0.173666
step789, Current loss: 0.162665
step790, Current loss: 0.119468
step791, Current loss: 0.170333
step792, Current loss: 0.131066
step793, Current loss: 0.182473
step794, Current loss: 0.216426
step795, Current loss: 0.16045
step796, Current loss: 0.343135
step797, Current loss: 0.170303
step798, Current loss: 0.118847
step799, Current loss: 0.204213
step800, Current loss: 0.181993
step801, Cu

step1027, Current loss: 0.144547
step1028, Current loss: 0.184427
step1029, Current loss: 0.157554
step1030, Current loss: 0.197179
step1031, Current loss: 0.203859
step1032, Current loss: 0.215977
step1033, Current loss: 0.202462
step1034, Current loss: 0.143059
step1035, Current loss: 0.232324
step1036, Current loss: 0.190865
step1037, Current loss: 0.148817
step1038, Current loss: 0.171641
step1039, Current loss: 0.132128
step1040, Current loss: 0.20935
step1041, Current loss: 0.225169
step1042, Current loss: 0.202801
step1043, Current loss: 0.142063
step1044, Current loss: 0.166838
step1045, Current loss: 0.164778
step1046, Current loss: 0.204574
step1047, Current loss: 0.145145
step1048, Current loss: 0.181458
step1049, Current loss: 0.15206
step1050, Current loss: 0.190282
step1051, Current loss: 0.155436
step1052, Current loss: 0.144259
step1053, Current loss: 0.195953
step1054, Current loss: 0.155421
step1055, Current loss: 0.216832
step1056, Current loss: 0.211573
step1057, Cu

...

step4256, Current loss: 0.0995515
step4257, Current loss: 0.139044
step4258, Current loss: 0.0782637
step4259, Current loss: 0.363709
step4260, Current loss: 0.128481
step4261, Current loss: 0.0838306
step4262, Current loss: 0.100362
step4263, Current loss: 0.142193
step4264, Current loss: 0.15314
step4265, Current loss: 0.0556979
step4266, Current loss: 0.096265
step4267, Current loss: 0.0914633
step4268, Current loss: 0.19159
step4269, Current loss: 0.116705
step4270, Current loss: 0.126274
step4271, Current loss: 0.0845734
step4272, Current loss: 0.142619
step4273, Current loss: 0.186441
step4274, Current loss: 0.0993948
step4275, Current loss: 0.174155
step4276, Current loss: 0.0573923
step4277, Current loss: 0.111054
step4278, Current loss: 0.100328
step4279, Current loss: 0.0921347
step4280, Current loss: 0.144774
step4281, Current loss: 0.0890786
step4282, Current loss: 0.139589
step4283, Current loss: 0.119426
step4284, Current loss: 0.0978119
step4285, Current loss: 0.149275
s

step4502, Current loss: 0.0984826
step4503, Current loss: 0.116703
step4504, Current loss: 0.0774841
step4505, Current loss: 0.0994766
step4506, Current loss: 0.0954434
step4507, Current loss: 0.066084
step4508, Current loss: 0.117149
step4509, Current loss: 0.0731856
step4510, Current loss: 0.080858
step4511, Current loss: 0.0995306
step4512, Current loss: 0.113336
step4513, Current loss: 0.0684768
step4514, Current loss: 0.127257
step4515, Current loss: 0.0732386
step4516, Current loss: 0.0742581
step4517, Current loss: 0.0627949
step4518, Current loss: 0.084413
step4519, Current loss: 0.0873566
step4520, Current loss: 0.0855439
step4521, Current loss: 0.0823786
step4522, Current loss: 0.091273
step4523, Current loss: 0.111844
step4524, Current loss: 0.0866895
step4525, Current loss: 0.0972888
step4526, Current loss: 0.0940469
step4527, Current loss: 0.0832749
step4528, Current loss: 0.0764043
step4529, Current loss: 0.0997349
step4530, Current loss: 0.0742752
step4531, Current loss:

step4748, Current loss: 0.0870803
step4749, Current loss: 0.0658987
step4750, Current loss: 0.11615
step4751, Current loss: 0.169108
step4752, Current loss: 0.143797
step4753, Current loss: 0.101862
step4754, Current loss: 0.110063
step4755, Current loss: 0.1081
step4756, Current loss: 0.0770873
step4757, Current loss: 0.0841547
step4758, Current loss: 0.0590179
step4759, Current loss: 0.0532133
step4760, Current loss: 0.100074
step4761, Current loss: 0.0881537
step4762, Current loss: 0.137969
step4763, Current loss: 0.0975843
step4764, Current loss: 0.0581597
step4765, Current loss: 0.152882
step4766, Current loss: 0.137847
step4767, Current loss: 0.0631321
step4768, Current loss: 0.0784819
step4769, Current loss: 0.122386
step4770, Current loss: 0.127121
step4771, Current loss: 0.0794829
step4772, Current loss: 0.114717
step4773, Current loss: 0.120512
step4774, Current loss: 0.0708983
step4775, Current loss: 0.100361
step4776, Current loss: 0.11415
step4777, Current loss: 0.0934311


step4993, Current loss: 0.097623
step4994, Current loss: 0.0815479
step4995, Current loss: 0.0945578
step4996, Current loss: 0.0919083
step4997, Current loss: 0.0938518
step4998, Current loss: 0.136712
step4999, Current loss: 0.142729
step5000, Current loss: 0.0716183
step5001, Current loss: 0.114603
step5002, Current loss: 0.112133
step5003, Current loss: 0.115381
step5004, Current loss: 0.0611518
step5005, Current loss: 0.119678
step5006, Current loss: 0.0514042
step5007, Current loss: 0.0679504
step5008, Current loss: 0.158585
step5009, Current loss: 0.07327
step5010, Current loss: 0.0480569
step5011, Current loss: 0.0537826
step5012, Current loss: 0.0948335
step5013, Current loss: 0.078754
step5014, Current loss: 0.136574
step5015, Current loss: 0.191522
step5016, Current loss: 0.102572
step5017, Current loss: 0.0706083
step5018, Current loss: 0.0812639
step5019, Current loss: 0.0803792
step5020, Current loss: 0.121825
step5021, Current loss: 0.0980788
step5022, Current loss: 0.077

step5238, Current loss: 0.0971157
step5239, Current loss: 0.0697462
